# Create Shipping manifest matching table


Several MTA: BioRep, IU, TA    

Create the table of all barcodes per persion. 

The table is compared with the summary table created previously

Table is stored.

In [1]:
import pandas as pd
import numpy as np
import os
import subprocess
import glob
from functools import partial
from os import chdir
import datetime
import csv

tmp = '/lscratch/' + os.environ['SLURM_JOB_ID'] + '/MTA'
if os.path.exists(tmp):
    print(tmp, 'exists')
else:
    os.mkdir(tmp)
    print('Make', tmp)

chdir(tmp)

/lscratch/21965786/MTA exists


In [2]:
!cp /data/LNG/iwakih2/dataset/PPMI/WGS/Shipping/* .

In [3]:
BioRepF1 = pd.read_excel('BioRep Full List PPMI 01252016.xlsx', sheet_name='To Be Aliquoted', dtype={'Specimen Bar Code':object})
BioRepF1.index = BioRepF1['Last Name']
BioRepF1 = BioRepF1[['Specimen Bar Code']]

BioRepF2 = pd.read_excel('BioRep Full List PPMI 01252016.xlsx', sheet_name='To Pull for Shipping', dtype={'Specimen Bar Code':object})
BioRepF2.index = BioRepF2['Last Name']
BioRepF2= BioRepF2[['Specimen Bar Code']]

BioRepM1 = pd.read_excel('BioRep Manifest Singleton PPMI DNA.xlsx', sheet_name='Sheet1', dtype={'Specimen bar code':object})
BioRepM1.index = BioRepM1['PPMI ID']
BioRepM1= BioRepM1[['Specimen bar code']]

BioRep=BioRepF1.join(BioRepF2, how='outer', lsuffix='BioRepF1', rsuffix='BioRepF2').join(BioRepM1, how='outer', rsuffix='BioRepM')
BioRep.to_csv("BioRep.csv")

In [4]:
IUF1 = pd.read_excel('IU Full List PPMI 01252016.xlsx', sheet_name='To Be Aliquoted', dtype={'Specimen Bar Code':object})
IUF1.index = IUF1['Last Name']
IUF1 = IUF1[['Specimen Bar Code']]

IUF2 = pd.read_excel('IU Full List PPMI 01252016.xlsx', sheet_name='To Pull for Shipping', dtype={'Specimen Bar Code':object})
IUF2.index = IUF2['Last Name']
IUF2= IUF2[['Specimen Bar Code']]

IUM1 = pd.read_excel('IU Manifest Singleton PPMI DNA.XLSX', sheet_name='Sheet1', dtype={'Specimen Bar Code':object})
IUM1.index = IUM1['PPMI ID']
IUM1= IUM1[['Specimen Bar Code']]

IU=IUF1.join(IUF2, how='outer', lsuffix='IUF1', rsuffix='IUF2').join(IUM1, how = 'outer', rsuffix='IUM')
IU.to_csv("IU.csv")

In [5]:
TAP1 = pd.read_excel('TA PPMI Singleton Feb 2017.xlsx', sheet_name='Sheet1', dtype={'Specimen Bar Code':object})
TAP1.index = TAP1['Last Name']
TAP1 = TAP1[['Specimen Bar Code']]

TAM1 = pd.read_excel('TA Manifest Singleton PPMI DNA.xlsx', sheet_name='Sheet1', dtype={'Specimen bar code':object})
TAM1.index = TAM1['PPMI ID']
TAM1= TAM1[['Specimen bar code']]

TA = TAP1.join(TAM1, how = 'outer', lsuffix='TA1', rsuffix='TAM')
TA.to_csv("TA.csv")

In [6]:
All = BioRep.join(IU, how = 'outer', lsuffix='BioRep', rsuffix='IU').join(TA, how = 'outer',lsuffix="_", rsuffix='TA')
All2 = All.iloc[All.index>0, :]
All2.to_csv("BioRep_IU_TA.csv")

In [7]:
# check the number of barcodes per person
ref = pd.read_csv("BioRep_IU_TA.csv", header=0, names=['v1', "v2", "v3", "v4", "v5", "v6", "v7", "v8"], dtype=object)
uniqIDs = ref.index[ref.index.duplicated()==False]
barcodes=[]
for ID in uniqIDs:
    t1 = ref[ref.index==ID]
    t2 =[i for i in t1.values.reshape(-1) if not pd.isnull(i)]
    uniq_bar = list(set(t2))
    fill = 5 - len(uniq_bar) # 5 slots are enough for one person's barcodes
    uniq_bar.extend([np.nan]*fill)
    barcodes.append(uniq_bar) 
barcodeT = pd.DataFrame(data = barcodes,
                  index = uniqIDs.astype('float32').astype('int').astype('object'),
                  columns = ['barcode1', 'barcode2', 'barcode3', 'barcode4', 'barcode5'])
barcodeT.notna().apply(sum, axis=0)

barcode1    1599
barcode2     221
barcode3      61
barcode4       0
barcode5       0
dtype: int64

In [8]:
# Drop ID4 and ID5 because they are unused slots.
barcodeT = barcodeT.drop(['barcode4', 'barcode5'], 1)

In [9]:
# Now read another FILE which was created previously (The summarized one)
FILE=pd.read_excel("Barcode_PPMI_NIA_AMPPD_ShipandSeq_Summary060118FINALC.xlsx", sheet_name='Annotated PPMI Subj List',  dtype=object)
FILE.index=FILE['MJFF Record']
Match = FILE.join(barcodeT, how='outer')
Match['PATNO'] = Match.index

In [10]:
## Matching barcode1-3 to ones in the summarized one
def giveStatus1(x):
    v = x[['barcode1', 'barcode2', 'barcode3']].isin([x['1st Shipment Barcodes']]) # if 1st Shipment Barcode is in Manifest
    if pd.isnull(x['1st Shipment Barcodes']):
        return "NoReference"
    elif sum(v) != 0: # exists
        match = [['barcode1', 'barcode2', 'barcode3'][i] for (i, x) in enumerate(v) if x]
        return "".join(match)
    elif pd.isnull(x['barcode1']): # not exists
        return 'NotInManifest'
    else:
        return 'Unmatch'
def giveStatus2(x):
    v = x[['barcode1', 'barcode2', 'barcode3']].isin([x['Second Shipment Barcodes']])
    if pd.isnull(x['Second Shipment Barcodes']):
        return "NoReference"
    elif sum(v) != 0:
        match = [['barcode1', 'barcode2', 'barcode3'][i] for (i, x) in enumerate(v) if x]
        return "".join(match)
    elif pd.isnull(x['barcode1']): # not exists
        return 'NotInManifest'
    else:
        return 'Unmatch'
Match['Match_1st'] = Match.apply(giveStatus1, axis=1)
Match['Match_2nd'] = Match.apply(giveStatus2, axis=1)
def giveStatusReport(x):
    if x['Match_1st'] == 'NotInManifest':
        return 'NotInManifest'
    elif sum(x[['Match_1st', 'Match_2nd']] == 'NoReference') ==2:
        if pd.notna(x['barcode1']):
            return 'OnlyInManifest'
        elif sum(pd.isnull(x[['1st Shipment Barcodes', 'Second Shipment Barcodes']]))<2:
            return 'NotInManifest'
        else:
            return 'NoBarcode'
    elif x['Match_1st'] == 'Unmatch':
        if x['Match_2nd'] == 'NoReference':
            return 'Unmatch'
        else:
            return '1stUnmatchBut2ndMatch'
    elif x['Match_2nd'] == 'Unmatch':
        if x['Match_1st'] == 'NoReference':
            return 'Unmatch'
        else:
            return '1stMatchBut2ndUnmatch'
    else:
        return 'Match'
Match['Report']=Match.apply(giveStatusReport, axis=1)

In [11]:
# Join with Fam file
df=pd.read_table('/data/LNG/PPMI_WGS/july_2018/PPMI_july2018.fam', header=None, 
                 sep=' ',
                 names = ['FID', 'IID','FAT', 'MAT','SEX', 'PHENO'])
ID_genotyped = [y for x,y in df.iloc[:,0].str.split('SI')]
ID_genotyped =list(map(int, ID_genotyped))
df.index = ID_genotyped
df.index = df.index.astype('object')
genotyped = df.drop(['FID', 'FAT', 'MAT','SEX', 'PHENO'], 1)
Match2 = Match.join(genotyped, how='outer')

/usr/local/Anaconda/envs/py3.5/lib/python3.5/site-packages/pandas/core/reshape/merge.py:765: RuntimeWarning: unorderable types: int() < str(), sort order is undefined for incomparable objects
  sort=self.sort)


In [12]:
# Give Reporting IDs for those in the genpotyped
def giveReportBCD(x):
    if pd.notnull(x['IID']):
        if x['Match_1st'] in ['barcode1', 'barcode2', 'barcode3']:
            BCD = x['Match_1st']
            ans = x[BCD]
        elif x['Match_2nd'] in ['barcode1', 'barcode2', 'barcode3']:
            BCD = x['Match_2nd']
            ans = x[BCD]
        else:
            ans = np.nan
    else:
        ans = np.nan
    return (ans)
Match2['Reporting_ID'] = Match2.apply(giveReportBCD, axis = 1)
Match2.to_csv("Matching.csv")

In [13]:
df = pd.read_excel("Barcode_PPMI_NIA_AMPPD_ShipandSeq_Summary060118FINALC.xlsx", 
                   sheet_name="PPMI smpls seq'd not on AMPPD",
                   header = None,
                   dtype=object)
t = df[0][pd.notna(df[0])]
NOSENT = [i for i in t.tolist() if 'PPMI' in i]
df_nosent = pd.DataFrame(data={'IID':NOSENT, 'Unshipped':'YES'})

In [14]:
df = pd.merge(Match2, df_nosent, on='IID', how='left')
df.to_csv('/data/LNG/iwakih2/dataset/PPMI/WGS/Shipping/MatchingTable.csv')

In [15]:
df.shape

(1820, 25)